<br></br>
# **나이브베이즈 모델의 활용**
1. **베이지안 평가 학습모델**을 불러온다 (0:부정 / 1:긍정)
1. **분석을 위한 네이버 뉴스 댓글**을 수집한다 (600개 내외가 적합)
1. **긍정/ 부정 댓글**을 분류한다
1. **긍정 리뷰**의 **Token** 을 트위터 **Stemming**으로 정규화 한다
1. **정규화된 Token** 중 **"명사", "동사", "형용사"** 만 수집
1. 수집된 Token List 객체를 **nltk.Text 로 변환 후 빈도상위 10개**를 출력한다

<img src="http://www.meconomynews.com/news/photo/201806/15255_14549_2116.png" />

<br></br>
## **1 모듈과 데이터 불러오기**
Import Data / Datum

In [1]:
# Step 1 : 앞에서 학습한 Classifier 모델을 불러온다
# >> './data/classifiers.model' --> classifier
import pickle
classifier = pickle.load(open('./data/classifiers.model', 'rb'))
classifier

In [2]:
# Step 2 : 분류기준 단어목록 불러오기
# >> './data/selected.words' --> selected_words
selected_words = pickle.load(open('./data/selected.words', 'rb'))
selected_words

['./Punctuation',
 '영화/Noun',
 '하다/Verb',
 '이/Josa',
 '보다/Verb',
 '의/Josa',
 '../Punctuation',
 '에/Josa',
 '가/Josa',
 '도/Josa',
 '을/Josa',
 '.../Punctuation',
 '은/Josa',
 '는/Josa',
 '들/Suffix',
 ',/Punctuation',
 '없다/Adjective',
 '를/Josa',
 '있다/Adjective',
 '좋다/Adjective',
 '되다/Verb',
 '이/Determiner',
 '너무/Noun',
 '재밌다/Adjective',
 '?/Punctuation',
 'ㅋㅋ/KoreanParticle',
 '정말/Noun',
 '한/Josa',
 '다/Josa',
 '적/Suffix',
 '것/Noun',
 '!/Punctuation',
 '으로/Josa',
 '같다/Adjective',
 '이/Noun',
 '않다/Verb',
 '아니다/Adjective',
 '진짜/Noun',
 '연기/Noun',
 '로/Josa',
 '점/Noun',
 '만/Josa',
 '나오다/Verb',
 '과/Josa',
 '평점/Noun',
 '~/Punctuation',
 '이렇다/Adjective',
 '만들다/Verb',
 '에서/Josa',
 '최고/Noun',
 '1/Number',
 '스토리/Noun',
 '이다/Verb',
 '왜/Noun',
 '인/Josa',
 '생각/Noun',
 '..../Punctuation',
 '이다/Josa',
 '드라마/Noun',
 '말/Noun',
 '감동/Noun',
 '배우/Noun',
 '사람/Noun',
 '안/Noun',
 '보고/Noun',
 '와/Josa',
 '더/Noun',
 '하고/Josa',
 '아깝다/Adjective',
 '거/Noun',
 '뭐/Noun',
 '재미있다/Adjective',
 '까지/Josa',
 '감독/Noun',
 '지루하다/Adj

In [3]:
# Step 3 : Tokenizer 생성/ 판단모듈
from konlpy.tag import Twitter
twitter = Twitter()

# 문단을 긍부정 분석용 Token 생성하기
def tokenize(doc):
    doc = ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

# 위에서 생성한 Token을 활용하여 판단하기
def classify_text(text):
    tokens = tokenize(text)             # 트위터로 Stemming/ Taggging 후 Selected_word 판단결과 객체
    return classifier.classify(tokens)  # 분류모델 평가

In [4]:
# Step 4 : 데이터와 모델, 함수를 활용하여 긍/부정 Test
# 0 : 부정모델 , 1 : 긍정모델

text   = """영화가 재미있어요"""
classify_text(text)

'1'

<br></br>
## **2 뉴스 데이터 댓글데이터 불러오기**
replies [참고](http://blog.naver.com/PostView.nhn?blogId=sangdo14&logNo=220610607049&categoryNo=0&parentCategoryNo=163&viewDate=&currentPage=1&postListTopCurrentPage=1&from=search)

In [5]:
# 외부뉴스 데이터 수집하기
from txtutil import Naver_news_rep
# news_url = "https://news.naver.com/main/hotissue/read.nhn?mid=hot&" +\
#            "sid1=&cid=1064721&iid=3227816&oid=001&aid=0010068060&ptype=021"
# replies  = Naver_news_rep(news_url)

# import pickle
# pickle.dump(replies, open('./data/naver_news.rep', 'wb'))

In [6]:
# 네이버 뉴스 댓글 저장데이터 불러오기
# >> './data/naver_news.rep' --> data

import pickle
data = pickle.load(open('./data/naver_news.rep', 'rb'))
print("Data length : {}\nData sample : {}".format(
    len(data), data[:5]))

Data length : 523
Data sample : ['"방수 된다고 했지, 고장 안난다고 한적 없습나다 고갱님^^"', '"휴지에 물 묻혀서 청소하는게 생활방수 아니면 뭐임??"', '"이러니 순실전자니 뭐니해도 삼성 쓸수밖에.AS가 넘사벽이잖아.왜 소비자가 불편함과 불이익을 감수하면서 까지 불매를 해야함?"', '"실제 애플 홈페이지에 소개된 제품 설명 맨 하단에 보면 작은 글씨로 \\"아이폰은 IP67 등급 생활 방수 및 방진 기능을 갖추고 있다\\"면서도 \\"액체에 의한 손상은 보증 대상이 아니다\\"라고 못 박고 있다. &lt;&lt;&lt; 이부분 어안벙벙하긴 함."', '"침수됐으니 유상으로 수리를 받아야 한다며 76만원을 내라고 했다...이거 실화냐 ? 이게 말이여 ? 방구여 ? ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ"']


<br></br>
## **Mission Test**
긍정 부정댓글 분류하기

In [7]:
# Mission 1
# 전체 댓글 갯수중 긍정/ 부정 갯수 계산하기

In [8]:
pro, con = [], []
for datum in data:
    if classify_text(datum) == "1":
        pro.append(datum)
    else:
        con.append(datum)

In [9]:
# 긍정 / 부정 분류된 데이터를 Json 과 같은 형태로 정리
result = {}
result["긍정"] = pro
result["부정"] = con
result

{'긍정': ['"방수 된다고 했지, 고장 안난다고 한적 없습나다 고갱님^^"',
  '"ㅋㅋㅋㅋ 물티슈로 닦았는데 침수피해 ㅋㅋㅋ 접시물에 코 박고 익사 ㄷㄷ"',
  '"휴지에물가지고 침수됐을리가..저건 100프로뻥임"',
  '"삼성 서비스는 인정. 진짜 헬지와 비교됨."',
  '"그래도 갓성전자네"',
  '"G6 변기에 몇번 빠뜨렸는데 침수로 인한 고장은 한번도 없었네요."',
  '"갤럭시S7 엣지 흐르는 물에 씻었는데\\n방수 잘 되네요"',
  '"방수랑 생활방수 구분못하는 저능아들이나 저지르는 짓이죠 생활방수는 말그대로 일상생활중 ( 빗물이나 물안에 모르고 잠깐 빠뜨렷을 시 ) 방수란말인데 그걸 수영장이나 물속에 쳐갖고들어가서 수중촬영쳐 하고 자빠져잇음 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ"',
  '"역시 삼성. 나는 앞으로도 삼성^^"',
  '"난 갤럭시 매주 한번씩 비누로 씻고 물로 헹구는데 몇년째 멀쩡한데?ㅋㅋ"',
  '"음 물에 양이 중요한듯한데요 ... 전 방수기능없는 아이폰3gs부터 아이폰4 마지막사용한 아이폰5s 후  갤럭시s8을 사용중인데 전 외출후 항상  물티슈로 앞뒤 다 닦는데 그런적이없었는데.... 아마 물을 엄청나게 묻히고 하신게 아닌지.....  모든지 적당히\\n.. 과유불급이라햇습니다."',
  '"입급완료 됐나보네..."',
  '"ㅋㅋㅋㅋㅋ대박 역시 애플"',
  '"샘숭이 어제 좋지 않은 기사 몇 개 뜨더니 마케팅 활발히 하네. 홍보실 일 참 열심히 한다"',
  '"그러니까 왜 아이폰을 사니?"',
  '"방수는 소니가 갑임 수중촬영 한적도있음ㅎㅎ"',
  '"삼성....윈!"',
  '"앱등이 호구들에겐 침수도 감성이면 괜찮아. ㅋㅋ"',
  '"Note fe  누수후,수리비 100나왔음.\\n강하게 항의 하니,무료."',
  '"좝스빠가 언제 기능보고샀냐.. 얼어죽을 감성타령이자나"',
  '"ㄱㅐ쓰레기 아이폰을 아직도 쓰는 등쉰들인 진정  호갱님! As서비스땜에 삼성으로 넘어온지어언 3년! 아이폰 서비스는 개 씨레기임

In [10]:
len(result["긍정"])

149

In [11]:
len(result["부정"])

374

In [12]:
# Mission 2
# 긍정 댓글을 구분하여 분류한 뒤
# 명사, 형용사, 동사 Token만 추출하기 (전처리 / Token의 예외처리는 미포함)

In [13]:
%whos

Variable         Type                    Data/Info
--------------------------------------------------
Naver_news_rep   function                <function Naver_news_rep at 0x7f4761e32400>
Twitter          type                    <class 'konlpy.tag._twitter.Twitter'>
classifier       NaiveBayesClassifier    <nltk.classify.naivebayes<...>object at 0x7f47980db400>
classify_text    function                <function classify_text at 0x7f4761e1d620>
con              list                    n=374
data             list                    n=523
datum            str                     "역사가들은 현시대를 친일파강점기라고 부릅니다<...>재인 탄핵과 김대중 부관참시에 동참해주세요."
pickle           module                  <module 'pickle' from '/u<...>lib/python3.6/pickle.py'>
pro              list                    n=149
result           dict                    n=2
selected_words   list                    n=4000
text             str                     영화가 재미있어요
tokenize         function                <function tokenize at 0x7f4761e1d

In [14]:
texts = result['부정']
texts[:3]

['"휴지에 물 묻혀서 청소하는게 생활방수 아니면 뭐임??"',
 '"이러니 순실전자니 뭐니해도 삼성 쓸수밖에.AS가 넘사벽이잖아.왜 소비자가 불편함과 불이익을 감수하면서 까지 불매를 해야함?"',
 '"실제 애플 홈페이지에 소개된 제품 설명 맨 하단에 보면 작은 글씨로 \\"아이폰은 IP67 등급 생활 방수 및 방진 기능을 갖추고 있다\\"면서도 \\"액체에 의한 손상은 보증 대상이 아니다\\"라고 못 박고 있다. &lt;&lt;&lt; 이부분 어안벙벙하긴 함."']

In [15]:
filtered_tokens = []
for text in texts:
    tokens = twitter.pos(text ,stem=True)
    tokens = [token[0]    for token in tokens    
                          if (token[1] in ['Noun','Adjective','Verb']) and (len(token[0]) > 1 )]
    filtered_tokens += tokens

In [16]:
# Mission 3
# nltk.Text() 객체를 활용하기
# 상위 빈도수 50인 Token을 추출하기

In [17]:
import nltk
pos_nltk = nltk.Text(filtered_tokens, name='긍정댓글')
pos_nltk.vocab().most_common(20)

[('하다', 238),
 ('방수', 160),
 ('되다', 129),
 ('삼성', 121),
 ('쓰다', 95),
 ('아이폰', 86),
 ('없다', 74),
 ('애플', 68),
 ('광고', 68),
 ('아니다', 59),
 ('침수', 53),
 ('닦다', 50),
 ('있다', 47),
 ('받다', 43),
 ('문제', 37),
 ('기사', 37),
 ('기능', 34),
 ('보다', 32),
 ('가다', 28),
 ('물티슈', 28)]